# In-class notebook: 2025-01-29

In this notebook, we will use a coin flip example to look at several important concepts of Bayesian statistics. 

This notebook is intended to support Chapter 5.2-5.4 of the textbook, and material is taken from the following scripts (from astroML):
* https://github.com/astroML/astroML_figures/blob/main/book_figures/chapter5/fig_odds_ratio_coin.py



## Odds ratio

Odds ratio for two models, $O_{21}$, describing coin tosses. We toss the coin N times and find k tosses are heads.
* Model 1: we a priori known heads probability equal to $b_*$
* Model 2: the probability of heads is unknown, with uniform prior 0-1

We look at two values of $b_*$. 

In [ ]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
from scipy import integrate

Write down the Bayesian odds ratio as a function of $N$, $k$ and $b_{*}$.

In [ ]:
# check out how to vectorize a function here: 
# https://www.programiz.com/python-programming/numpy/vectorization
# https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html

@np.vectorize
def odds_ratio(n, k, bstar):
    """Odds ratio between M_2, where the heads probability is unknown,
    and M_1, where the heads probability is known to be `bstar`, evaluated
    in the case of `k` heads observed in `n` tosses.

    Eqn. 5.25 in the text
    """
    factor = 1. / (bstar ** k * (1 - bstar) ** (n - k))
    f = lambda b: b ** k * (1 - b) ** (n - k)

    return factor * integrate.quad(f, 0, 1)[0]

In [ ]:
fig = plt.figure(figsize=(8, 4))

subplots = [121, 122]
linestyles = ['-k', '--b']

# Look at two cases of N, two cases of b*
n_array = [10, 20]
bstar_array = [0.5, 0.1]

for subplot, n in zip(subplots, n_array):
    ax = fig.add_subplot(subplot, yscale='log')
    k = np.arange(n + 1)

    # plot curves for two values of bstar
    for ls, bstar in zip(linestyles, bstar_array):
        ax.plot(k, odds_ratio(n, k, bstar), ls,
                label=r'$b^* = %.1f$' % bstar)

    if subplot == 121:
        ax.set_ylabel(r'$O_{21}$')
        ax.legend(loc=2)
    else:
        ax.yaxis.set_major_formatter(plt.NullFormatter())

    ax.set_xlabel('$k$')
    ax.set_title('$N = %i$' % n)
    ax.set_ylim(8E-2, 1E3)
    ax.xaxis.set_major_locator(plt.MultipleLocator(n / 5))
    ax.grid()


Note that the odds ratio is minimized and below 1 (model 1 wins) when $k = b_* N$.

Let us think about some concrete cases:

* Assume we tossed 10 times ($N=10$), observer 1 heads ($k=1$), then the odds ratio of a model with unknown $b$ over a model with $b_*=0.1$ is small ~0.3 -- data does not strongly prefer the model with unknown $b$. OTOH, the odds ratio of a model with unknown $b$ over a model with $b_*=0.5$ is larger ~10 -- data has decent support for Model 2. 

* Assume we tossed 20 times ($N=20$), observer 8 heads ($k=8$), then the odds ratio of a model with unknown $b$ over a model with $b_*=0.1$ is very high ~100 -- data does strongly prefer the model with unknown $b$. OTOH, the odds ratio of a model with unknown $b$ over a model with $b_*=0.5$ is small ~0.4 -- data does not strongly prefer Model 2. 

## Hypothesis testing

Turning the above example into a hypothesis testing question: 

Say we throw the dice 20 times and observe 16 heads, we assume the null hypothesis to be that the coin is fair ($b_*=0.5$), and want to know if we can reject that hypothesis. 

### Classical approach

Plug $b=b_*=0.5$ into the binomial distribution, we get the mean $k^0=b_* N=10$, with $\sigma_k=\sqrt{Nb_*(1-b_*)} = 2.24$, so $k=16$ is about 2.7$\sigma_k$ away from the mean. If we set our criteria to reject to reject the null at $\alpha=0.05$, $k=16$ will be rejected.  

### Bayesian approach

We cannot reject the hypothesis without an alternative model. Using Model 2 as an alternative (considering all possible $b$ values), at $N=20$, $k=16$, the odds ratio is about 10.